# Data Processing

In this notebook you will find code for CSV processing. The raw CSV is formatted like this:

<img src="mRNA_raw_csv.png" width="400">

Column labels represent <font color = 'red'> <b> patients </b> </font> with either mild or severe RSV. Row labels represent <font color = 'red'> <b> genes </b> </font>. For each observation, we have a <font color = 'red'> <b> non-normalized mRNA count </b> </font> of that gene in each patient.

Let's start by importing our packages, then loading in our data.

In [47]:
# Imports
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

'''
Loaded RNA seq data, reformatted the gene names, and replaced case numbers with GSM numbers to make it easier to compare
with patient data.
'''
pd.set_option('display.max_columns', None) # We'd like to be able to view all columns

# raw counts
rna_seq = pd.read_csv('../RAW_DATA/GSE155925_Raw_counts_matrix.txt.gz', sep = '\t', header = 0, index_col = 0)

# patient data
patient_data = pd.read_csv("../RAW_DATA/patient_data.csv")
patient_data.index = patient_data['Accession']

# remove emsembl ID from gene names
def strip_name(gene):
    return gene[:gene.find(':')]

rna_seq = rna_seq.rename(index = lambda s: strip_name(s))


# rename cases by GSM number (raw counts data labels cases as "Case 1, Case 2, etc.")
gsm = []
for i in range(len(rna_seq.columns)):
    gsm.append('GSM' + str(4715941 + i))    
rna_seq.columns = gsm

# Filter out genes where mean mRNA count is less than 10
rna_seq = rna_seq[rna_seq.mean(axis = 1) > 10]

print(rna_seq.shape)
display(rna_seq.head())

(13994, 64)


,GSM4715941,GSM4715942,GSM4715943,GSM4715944,GSM4715945,GSM4715946,GSM4715947,GSM4715948,GSM4715949,GSM4715950,GSM4715951,GSM4715952,GSM4715953,GSM4715954,GSM4715955,GSM4715956,GSM4715957,GSM4715958,GSM4715959,GSM4715960,GSM4715961,GSM4715962,GSM4715963,GSM4715964,GSM4715965,GSM4715966,GSM4715967,GSM4715968,GSM4715969,GSM4715970,GSM4715971,GSM4715972,GSM4715973,GSM4715974,GSM4715975,GSM4715976,GSM4715977,GSM4715978,GSM4715979,GSM4715980,GSM4715981,GSM4715982,GSM4715983,GSM4715984,GSM4715985,GSM4715986,GSM4715987,GSM4715988,GSM4715989,GSM4715990,GSM4715991,GSM4715992,GSM4715993,GSM4715994,GSM4715995,GSM4715996,GSM4715997,GSM4715998,GSM4715999,GSM4716000,GSM4716001,GSM4716002,GSM4716003,GSM4716004
TSPAN6,7,12,7,7,5,6,5,3,3,4,41,17,2,28,25,44,58,4,2,5,7,6,3,0,7,4,11,11,5,5,3,11,8,5,8,19,10,5,6,21,9,16,7,11,5,53,10,18,22,8,12,27,9,4,18,3,60,34,22,37,1,18,6,29
DPM1,227,201,259,224,154,143,257,248,133,147,810,353,333,125,164,367,351,199,253,220,325,270,392,433,313,425,431,354,284,361,348,607,407,249,320,374,514,307,505,442,709,586,455,606,519,543,321,400,454,476,336,285,386,484,428,211,76,312,307,391,172,333,194,263
SCYL3,500,635,454,643,473,309,519,585,413,333,1323,1013,964,339,330,695,617,570,725,677,634,624,541,558,503,705,749,705,497,540,514,890,622,601,791,641,654,688,754,818,882,998,873,914,882,1343,899,1111,1213,1294,1026,567,993,1059,1058,540,295,892,820,1011,378,759,397,834
C1orf112,158,174,173,157,134,131,144,219,116,79,393,242,323,102,106,218,202,167,267,270,341,186,215,412,117,299,227,228,194,146,217,322,230,177,324,226,274,273,251,315,379,317,306,313,235,482,283,371,372,384,262,208,248,363,361,134,105,317,304,341,128,201,174,209
FGR,4723,1925,4931,10412,6132,3896,4142,4080,2408,3355,4505,11083,8909,3411,2203,3504,5208,7497,8619,4944,4884,7127,2956,2763,5713,4106,5945,5247,6825,8067,3317,12291,7987,9422,7154,5977,4805,5964,8811,6989,11269,8919,6361,10061,11220,5308,6174,5757,6331,6466,7375,2746,6846,7845,4402,14466,718,3665,4426,6031,5007,8914,3905,6426


In [48]:
'''
Add rows to include patient data
'''
# pathogen groups: 0 = negative, 1 = RSV, 2 = other
# pulled manually from site
groups = [0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 2, 0, 1, 2, 0, 2, 0, 2, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1]

rna_seq.loc[len(rna_seq.index)] = groups


# sex: 0 = M, 1 = F
# pulled manually from site
sex = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0]

rna_seq.loc[len(rna_seq.index)] = sex

# severity of RSV: 0 = patient does not have RSV
severity = [None] * 64
rna_seq.loc[len(rna_seq.index)] = severity

#rename the patient data rows
rna_seq = rna_seq.rename({13994: 'Group', 13995: 'Sex', 13996: 'Severity'})

# assign severity
for col in rna_seq.columns:
    if col in patient_data.index:
        rna_seq.at['Severity', col] = patient_data.at[col, 'Severity']


# display patient data
display(rna_seq.tail(5))

,GSM4715941,GSM4715942,GSM4715943,GSM4715944,GSM4715945,GSM4715946,GSM4715947,GSM4715948,GSM4715949,GSM4715950,GSM4715951,GSM4715952,GSM4715953,GSM4715954,GSM4715955,GSM4715956,GSM4715957,GSM4715958,GSM4715959,GSM4715960,GSM4715961,GSM4715962,GSM4715963,GSM4715964,GSM4715965,GSM4715966,GSM4715967,GSM4715968,GSM4715969,GSM4715970,GSM4715971,GSM4715972,GSM4715973,GSM4715974,GSM4715975,GSM4715976,GSM4715977,GSM4715978,GSM4715979,GSM4715980,GSM4715981,GSM4715982,GSM4715983,GSM4715984,GSM4715985,GSM4715986,GSM4715987,GSM4715988,GSM4715989,GSM4715990,GSM4715991,GSM4715992,GSM4715993,GSM4715994,GSM4715995,GSM4715996,GSM4715997,GSM4715998,GSM4715999,GSM4716000,GSM4716001,GSM4716002,GSM4716003,GSM4716004
WDFY4,594,667,880,1805,1473,1129,741,906,1033,997,988,1667,914,195,304,540,337,2809,2290,2976,898,1658,1070,1013,1515,1797,2137,1680,1575,2025,1103,2468,2563,1450,1535,1872,808,1354,2128,1839,1957,2230,1013,2413,1824,787,521,794,1118,1262,912,451,2222,1510,1742,3173,358,1614,868,1577,954,1080,1530,1414
RP3-454G6.2,16,9,4,7,10,2,4,18,6,2,18,10,6,26,41,24,27,23,19,22,13,12,5,8,9,13,20,15,10,8,10,21,16,12,16,10,11,13,15,16,17,25,16,11,19,19,9,15,6,14,14,22,14,14,14,7,20,10,10,18,6,15,4,9
Group,0,1,1,2,0,0,1,1,0,2,1,1,1,1,2,1,1,2,2,2,1,1,1,1,1,1,2,1,2,1,1,1,1,0,1,2,1,2,0,1,2,0,2,0,2,1,1,1,1,1,0,0,2,1,1,1,1,2,1,2,2,1,1,1
Sex,0,1,1,0,1,0,1,0,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,1,1,1,0,0,0,0,1,0
Severity,None,mild,mild,None,None,None,severe,mild,None,None,mild,mild,mild,severe,None,severe,mild,None,None,None,mild,mild,mild,mild,mild,mild,None,mild,None,mild,mild,severe,mild,None,severe,None,mild,None,None,mild,None,None,None,None,None,mild,mild,mild,mild,mild,None,None,None,mild,mild,severe,mild,None,mild,None,None,severe,severe,mild


In [51]:
# Calculate sum of each column, convert to proportions. 
def logarithm_2(element):
    if element > 0:
        return math.log(element) / math.log(2)
    else:
        return 0

# get rid of patient data rows
norm_rna_seq = rna_seq.drop(index = ['Group', 'Sex', 'Severity'])

# Normalize by total mRNA content per patient.
for label in norm_rna_seq.columns.tolist():
    norm_rna_seq[label] = norm_rna_seq[label].astype(int)
    col_sum = norm_rna_seq[label].sum()
    norm_rna_seq[label] = norm_rna_seq[label].div(col_sum)
    norm_rna_seq[label] = norm_rna_seq[label].apply(logarithm_2) # Take log proportion to avoid really small numbers.

# Let's weed out genes with low mRNA signal!
# First, let's define a function to take the average mRNA signal for a given gene, EXCLUDING ZEROES.
def mRNA_avg(row):
    num_nonzero = (row != 0).sum()
    
    if num_nonzero == 0:
        return -10000 # This value indicates the gene wasn't present in any patient
    
    return row.sum()/num_nonzero

# Apply to every row of dataframe to generate new column
norm_rna_seq['avg_log_mRNA'] = norm_rna_seq.apply(mRNA_avg, axis = 1)

# Sort by descending avg_log_mRNA
norm_rna_seq.sort_values('avg_log_mRNA', ascending = False, inplace = True)

# Let's drop the bottom 25% of genes. 
norm_rna_seq = norm_rna_seq.drop(index = norm_rna_seq.index[- round(norm_rna_seq.shape[0] * 0.25):])

# add patient data rows back
norm_rna_seq = norm_rna_seq.append(rna_seq.tail(3))
 
# Sort patients (columns) by severity
controls = []
mild = []
severe = []
for patient in norm_rna_seq.columns:
    if norm_rna_seq.loc['Severity', patient] == None:
        controls.append(patient)
    elif norm_rna_seq.loc['Severity', patient] == 'mild':
        mild.append(patient)
    else:
        severe.append(patient)

norm_rna_seq = norm_rna_seq.reindex(columns = controls + mild + severe)

# Final CSV
print(norm_rna_seq.shape)
norm_rna_seq.tail(20)

(10496, 65)


/var/folders/wk/_2tg02yn5wng2c8tsg5s0tv80000gn/T/ipykernel_35248/3497212210.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  norm_rna_seq = norm_rna_seq.append(rna_seq.tail(3))


,GSM4715941,GSM4715944,GSM4715945,GSM4715946,GSM4715949,GSM4715950,GSM4715955,GSM4715958,GSM4715959,GSM4715960,GSM4715967,GSM4715969,GSM4715974,GSM4715976,GSM4715978,GSM4715979,GSM4715981,GSM4715982,GSM4715983,GSM4715984,GSM4715985,GSM4715991,GSM4715992,GSM4715993,GSM4715998,GSM4716000,GSM4716001,GSM4715942,GSM4715943,GSM4715948,GSM4715951,GSM4715952,GSM4715953,GSM4715957,GSM4715961,GSM4715962,GSM4715963,GSM4715964,GSM4715965,GSM4715966,GSM4715968,GSM4715970,GSM4715971,GSM4715973,GSM4715977,GSM4715980,GSM4715986,GSM4715987,GSM4715988,GSM4715989,GSM4715990,GSM4715994,GSM4715995,GSM4715997,GSM4715999,GSM4716004,GSM4715947,GSM4715954,GSM4715956,GSM4715972,GSM4715975,GSM4715996,GSM4716002,GSM4716003,avg_log_mRNA
PSTK,-18.624568,-18.703541,-17.850639,-17.667545,-16.959816,-17.39889,-16.059199,-17.192849,-17.336306,-16.866943,-17.489399,-16.831626,-17.740228,-16.916276,-17.348359,-17.53649,-17.68571,-17.024515,-17.378667,-17.573754,-18.549027,-17.673055,-16.627077,-17.949307,-16.856165,-17.248197,-17.992384,-16.841487,-17.637817,-17.250424,-17.056885,-17.691165,-17.866,-17.26024,-17.538779,-17.035314,-16.895161,-17.881773,-18.130647,-17.545819,-17.119073,-17.717604,-17.536129,-17.161191,-18.042556,-17.062205,-17.504109,-17.470205,-17.660529,-17.892177,-17.38954,-17.561277,-16.933946,-15.411023,-17.264139,-17.22417,-18.510515,-16.679769,-16.760305,-17.990186,-17.351332,-18.161561,-18.14889,-17.549365,-17.434591
DOHH,-17.30264,-17.933023,-17.418922,-16.809564,-17.796318,-17.661924,-17.350965,-19.000204,-18.187783,-18.746649,-17.157323,-18.416589,-17.481493,-16.809361,-16.953221,-17.470901,-17.55106,-17.56248,-16.948032,-17.31072,-18.334015,-16.768915,-17.701077,-18.627379,-17.006725,-18.282144,-17.662235,-16.645936,-16.536279,-16.620792,-17.668614,-17.312654,-16.643608,-18.461874,-16.56387,-17.526702,-17.321976,-15.996637,-16.545685,-16.731831,-17.384967,-17.179947,-17.588596,-17.108242,-16.436835,-16.950312,-16.496913,-17.050096,-17.107988,-17.440481,-17.171117,-17.658574,-16.910487,-17.079401,-17.283768,-17.300119,-17.485853,-18.564292,-18.479123,-17.182831,-17.72217,-17.846059,-19.701431,-17.888167,-17.435092
SPTBN5,-17.954717,-18.703541,-16.431413,-18.419617,-18.903233,-17.746813,-15.749514,-17.415242,-16.235612,-16.557615,-17.028425,-16.675507,-17.190031,-17.263079,-16.207266,-16.985475,-17.175849,-17.320405,-16.535906,-16.904727,-16.93853,-17.48041,-17.425443,-18.276882,-16.936336,-17.334612,-17.729349,-17.473755,-18.178385,-18.024148,-17.934894,-18.340668,-18.386832,-18.011212,-16.842171,-17.728336,-16.538468,-17.104166,-17.060258,-17.196234,-16.90692,-17.890441,-16.714127,-16.664253,-17.683475,-16.667673,-17.826037,-18.2624,-17.823258,-18.307214,-18.159243,-18.303479,-17.560902,-16.028775,-18.047328,-17.602682,-18.324102,-16.592306,-17.894161,-17.132205,-17.471626,-17.746524,-18.250769,-17.653702,-17.435292
CTC-432M15.3,0.0,0.0,0.0,-17.458092,0.0,-19.331775,0.0,0.0,0.0,0.0,-17.359631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-17.066362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-16.822419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-22.452576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.003307,0.0,0.0,-15.520208,0.0,0.0,-15.910487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-17.436095
UGT8,-17.592147,-18.433452,-18.576464,-17.967105,-17.31827,-17.746813,-16.654357,-17.337239,-17.804455,-16.866943,-17.523751,-18.054018,-17.357758,-18.501238,-17.290256,-17.470901,-17.92387,-17.175457,-17.948032,-17.228926,-17.964065,-16.813654,-16.221084,-16.214598,-17.441128,-17.045105,-19.536704,-16.287889,-17.60807,-17.393382,-17.786795,-17.746307,-16.92171,-17.26024,-17.514117,-18.728336,-17.04326,-17.306658,-17.671216,-17.252088,-17.995925,-17.789157,-17.13403,-18.287948,-17.478655,-17.749615,-18.47554,-16.193687,-16.413614,-16.134149,-16.535391,-17.062471,-16.597896,-16.158836,-16.484908,-18.22417,-17.753786,-16.8053,-16.816158,-18.008334,-17.798791,-17.524131,-18.053732,-17.97563,-17.437183
AIFM3,-17.061632,-18.433452,-16.647547,-17.667545,-17.959816,-17.661924,-16.961019,-17.142

Perfect! Let's write this into a CSV.

In [52]:
norm_rna_seq.to_csv("normalized_mRNA_counts.csv", index = True)

In [54]:
'''
Data frames for patient data groups
'''
# RSV and control groups
control = norm_rna_seq.loc[:, norm_rna_seq.loc['Group'] == 0].drop(index = ['Group', 'Sex', 'Severity'])
rsv = norm_rna_seq.loc[:, norm_rna_seq.loc['Group'] == 1].drop(index = ['Group', 'Sex', 'Severity'])

control.to_csv('control_norm.csv', index = True)
rsv.to_csv('rsv_norm.csv', index = True)

# Severity
mild = norm_rna_seq.loc[:, norm_rna_seq.loc['Severity'] == 'mild'].drop(index = ['Group', 'Sex', 'Severity'])
severe = norm_rna_seq.loc[:, norm_rna_seq.loc['Severity'] == 'severe'].drop(index = ['Group', 'Sex', 'Severity'])

mild.to_csv('mild_norm.csv', index = True)
severe.to_csv('severe_norm.csv', index = True)

# check number of samples in each group
print(len(control.columns), len(rsv.columns), len(mild.columns), len(severe.columns))
print(mild.shape, severe.shape)
display(control.tail(3))

10 37 29 8
(10493, 29) (10493, 8)


,GSM4715941,GSM4715945,GSM4715946,GSM4715949,GSM4715974,GSM4715979,GSM4715982,GSM4715984,GSM4715991,GSM4715992
ZSCAN20,-18.129803,-17.210814,-17.023689,-17.177408,-17.932873,-17.91836,-17.905368,-18.151521,-17.736249,-16.44121
ISOC2,-16.954717,-17.522016,-15.79718,-17.433748,-17.992994,-18.503323,-18.619064,-18.027786,-16.673055,-16.556687
KIAA1671,-18.440143,-16.884586,-16.438726,-17.51621,-18.689602,-19.318898,-17.548674,-18.6029,-17.57352,-15.739552
